In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd 
import nltk
import functools
import operator

In [2]:
AUTHORS_SENTENCES = '/home/mibo/Datasets/comments_last_100_min_20com_per_auth_w_wc_10_200.csv'
OUTPUT_EMBEDDINGS_LOCATION = '/home/mibo/Datasets/embedded_comments_sl_3_last_100_min_20com_per_auth_w_wc_10_200'

In [54]:
def create_meta_data(tokenized_sentences, df):
    authors = []
    ids = []
    parts = []
    for sentence, (index, row) in zip(tokenized_sentences, df.iterrows()): 
        sent_count = len(sentence)
        authors.extend([row['author']]*sent_count)
        ids.extend([row['id']]*sent_count)
        parts.extend([ f"{elem}/{sent_count}" for elem in range(1, sent_count+1)])
    return authors, ids, parts 

create_meta_data(tokenized_sentences[0:5],authors_sentences[0:5])

(['icebrotha',
  'Independent',
  'Independent',
  'Independent',
  'Independent',
  'Independent',
  'Independent',
  'Independent',
  '_nephilim_',
  '_nephilim_',
  'flutterfly28',
  'flutterfly28',
  'thissistheN',
  'thissistheN'],
 ['em6m3hy',
  'em6m284',
  'em6m284',
  'em6m284',
  'em6m284',
  'em6m284',
  'em6m284',
  'em6m284',
  'em6m0vu',
  'em6m0vu',
  'em6m0jt',
  'em6m0jt',
  'em6m01u',
  'em6m01u'],
 ['1/1',
  '1/7',
  '2/7',
  '3/7',
  '4/7',
  '5/7',
  '6/7',
  '7/7',
  '1/2',
  '2/2',
  '1/2',
  '2/2',
  '1/2',
  '2/2'])

In [4]:
model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

In [5]:
authors_sentences = pd.read_csv(AUTHORS_SENTENCES)

In [26]:
%%time
tokenized_sentences = authors_sentences['body'].apply(lambda comment : [sent.lower() for sent in nltk.sent_tokenize(comment)])

CPU times: user 1min 14s, sys: 288 ms, total: 1min 14s
Wall time: 1min 14s


In [8]:
sentences_ready_for_bert = functools.reduce(operator.iconcat, tokenized_sentences, [])

In [57]:
len(sentences_ready_for_bert)

2636687

In [55]:
authors, ids, parts = create_meta_data(tokenized_sentences, authors_sentences)

In [58]:
%%time
step = 1000
header = True
with open(OUTPUT_EMBEDDINGS_LOCATION, 'a') as f:
    for chunk in range(0, len(sentences_ready_for_bert), step):
        print(f"Processing chunk: {chunk}")
        encoded_sentences = model.encode(sentences_ready_for_bert[chunk: chunk+step])
        df = pd.DataFrame(encoded_sentences)
        df['author'] = authors[chunk : chunk+step]
        df['ids'] = ids[chunk : chunk+step]
        df['parts'] = parts[chunk : chunk+step]
        df.to_csv(f, header=header)
        header = False
        break

Processing chunk: 0
CPU times: user 5min 18s, sys: 384 ms, total: 5min 18s
Wall time: 1min 20s


1024

In [42]:
a = pd.read_csv(OUTPUT_EMBEDDINGS_LOCATION)

/home/mibo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1027) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
a

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1017,1018,1019,1020,1021,1022,1023,author,ids,parts
0,0.0,-0.250850,0.017321,0.208074,0.198622,0.100934,0.202161,0.187587,0.875983,-1.466893,...,0.407247,-0.014469,-0.776118,0.423684,0.186938,-0.624432,-0.651650,icebrotha,em6m3hy,"range(0, 1)"
1,1.0,1.970671,-0.142992,0.163765,0.261987,-0.591412,-0.091519,0.943089,-0.047490,0.278959,...,-0.511276,0.271308,0.472187,-0.680771,-0.499429,0.180990,-0.209845,Independent,em6m284,"range(0, 7)"
2,2.0,0.979489,0.061023,0.393305,-0.525289,-0.202907,-0.346870,0.139364,0.708136,-0.363207,...,-0.750430,-0.679599,0.439151,0.173733,-0.365304,-0.163899,-0.275633,Independent,em6m284,"range(0, 2)"
3,3.0,-0.910160,-1.134951,-0.092138,-0.121053,-0.615404,0.843918,0.672476,0.087172,0.102676,...,-0.608531,-0.343523,-0.349959,1.104170,-0.352771,0.447795,-0.208806,Independent,em6m284,"range(0, 2)"
4,4.0,0.579423,-0.810361,-0.087501,0.397510,0.123815,0.173381,-0.319881,-0.276588,1.142920,...,-0.128489,-0.502123,-0.335288,-0.967347,0.612375,0.939741,0.763273,Independent,em6m284,"range(0, 2)"
5,5.0,0.215521,0.672253,0.605101,-0.062294,-0.238088,0.265157,-0.151617,0.889373,-0.458541,...,0.169416,-1.008229,-0.325667,0.476806,0.082992,-0.330788,0.484948,Independent,em6m284,"range(0, 4)"
6,6.0,0.215521,0.672253,0.605101,-0.062294,-0.238088,0.265157,-0.151617,0.889373,-0.458541,...,0.169416,-1.008229,-0.325667,0.476806,0.082992,-0.330788,0.484948,Independent,em6m284,"range(0, 2)"
7,7.0,0.270990,0.844984,-0.114110,-0.184707,-0.757048,-0.731870,0.324315,-0.726303,-0.721834,...,0.062938,0.365250,0.049924,0.734583,-0.605337,0.031891,-0.791090,Independent,em6m284,"range(0, 2)"
8,8.0,-0.400079,-1.394377,0.676293,-0.115249,-0.324694,0.843960,-1.113299,0.955834,0.109418,...,0.332270,0.749803,-0.796637,-0.135425,-0.420935,-0.677047,-0.042275,_nephilim_,em6m0vu,"range(0, 3)"
9,9.0,1.073301,-0.005062,0.977566,0.787292,-0.823544,1.034223,0.187533,-0.318973,0.226637,...,-0.321218,0.098633,-0.638133,-0.483759,-0.554898,-0.296102,-0.567802,_nephilim_,em6m0vu,"range(0, 1)"


In [117]:
 ex = nltk.sent_tokenize('Dobar dan moje ime je Nikola. Kako ste?. Niko nije kao ti? Ja sam Radovan!. Kebab?')
merge_sentences(ex, to_lower_case)

['dobar dan moje ime je nikola. kako ste?. niko nije kao ti?',
 'kako ste?. niko nije kao ti? ja sam radovan!.',
 'niko nije kao ti? ja sam radovan!. kebab?']

['This framework generates embeddings for each input sentence. Sentences are passed as a list of string. The quick brown fox jumps over the lazy dog.']